# Combinar conjuntos de datos: Concat y Append

Algunos de los estudios de datos más interesantes proceden de la combinación de distintas fuentes de datos.
Estas operaciones pueden implicar cualquier cosa, desde una concatenación muy directa de dos conjuntos de datos diferentes, hasta uniones y fusiones más complicadas al estilo de las bases de datos que manejan correctamente cualquier solapamiento entre los conjuntos de datos.
``Series`` y ``DataFrame`` se construyen con este tipo de operaciones en mente, y Pandas incluye funciones y métodos que hacen que este tipo de manipulación de datos sea rápida y sencilla.

Aquí echaremos un vistazo a la simple concatenación de ``Series`` y ``DataFrame``s con la función ``pd.concat``; más tarde nos sumergiremos en fusiones y uniones en memoria más sofisticadas implementadas en Pandas.

Comenzamos con las importaciones estándar:

In [1]:
import pandas as pd
import numpy as np

> #### **FUNCIÓN RÁPIDA PARA CREAR DATAFRAMES = `def make_df(cols, ind):`**

Por conveniencia, definiremos esta función que crea un ``DataFrame`` de una forma particular que será útil más adelante:

In [2]:
def make_df(cols, ind):
    """Crea rápidamente un DataFrame"""
    data = {c: [str(c) + str(i) for i in ind]
            for c in cols}
    return pd.DataFrame(data, ind)

# ejemplo DataFrame
make_df('ABC', range(3))

,A,B,C
0,A0,B0,C0
1,A1,B1,C1
2,A2,B2,C2


> #### **CLASE PARA MOSTRAR DATAFRAMES UNO AL LADO DEL OTRO = `class display(object):`**

Además, crearemos una clase rápida que nos permita mostrar múltiples ``DataFrame`` uno al lado del otro. El código hace uso del método especial ``_repr_html_``, que IPython utiliza para implementar su visualización de objetos enriquecidos:

In [3]:
class display(object):
    """Mostrar la representación HTML de varios objetos"""
    template = """<div style="float: left; padding: 10px;">
    <p style='font-family:"Courier New", Courier, monospace'>{0}</p>{1}
    </div>"""
    def __init__(self, *args):
        self.args = args
        
    def _repr_html_(self):
        return '\n'.join(self.template.format(a, eval(a)._repr_html_())
                         for a in self.args)
    
    def __repr__(self):
        return '\n\n'.join(a + '\n' + repr(eval(a))
                           for a in self.args)
    

Su utilidad quedará más clara a medida que avancemos en el siguiente apartado.

- **ANTES** = Se utilizaba `.append()` para concatenar de la siguiente manera: 

Teniamos 2 DataFrames para concatenar= df1 y df2 

`df1.append(df2)` > Con esto, sabíamos que le agregabamos al df1, el df2 y ya quedaba así. 

- **AHORA** = Se utiliza `pd.concat([ ])` y tenes que guardarlo en una variable para que el NUEVO DataFrame te quede guardado. 

 `df_concatenado = pd.concat([df1,df2])` 


>>> ## **Recall: Concatenación de matrices NumPy**

La concatenación de objetos ``Series`` y ``DataFrame`` es muy similar a la concatenación de arrays Numpy, que se puede hacer a través de la función ``np.concatenate`` como se discutió en [The Basics of NumPy Arrays](02.02-The-Basics-Of-NumPy-Arrays.ipynb).
Recuerda que con ella puedes combinar el contenido de dos o más arrays en un único array:

In [4]:
x = [1, 2, 3]
y = [4, 5, 6]
z = [7, 8, 9]
np.concatenate([x, y, z])

array([1, 2, 3, 4, 5, 6, 7, 8, 9])

El primer argumento es una lista o tupla de matrices a concatenar.
Además, toma una palabra clave ``axis`` que permite especificar el eje a lo largo del cual se concatenará el resultado:

In [5]:
x = [[1, 2],
     [3, 4]]
np.concatenate([x, x], axis=1)

array([[1, 2, 1, 2],
       [3, 4, 3, 4]])

>>> ## **Concatenación simple con ``pd.concat()``**

``pd.concat()`` can be used for a simple concatenation of ``Series`` or ``DataFrame`` objects, just as ``np.concatenate()`` can be used for simple concatenations of arrays:
Pandas tiene una función, ``pd.concat()``, que tiene una sintaxis similar a ``np.concatenate`` pero contiene una serie de opciones que discutiremos momentáneamente:

```python
# Firma en Pandas v0.18
pd.concat(objs, axis=0, join='outer', join_axes=None, ignore_index=False,
          keys=None, levels=None, names=None, verify_integrity=False,
          copy=Verdadero)
```

``pd.concat()`` puede utilizarse para una simple concatenación de objetos ``Series`` o ``DataFrame``, al igual que ``np.concatenate()`` puede utilizarse para simples concatenaciones de arrays:

>> #### **``pd.concat()`` + `.reset_index(drop=True)`**

* `pd.concat()` = Concatena series donde cada una contiene sus propios índices
* `.reset_index()` = Resetea los índices y los transforma en columnas. Crea columnas adicionales que contienen los índices anteriores, y a las filas les pone el índice ímplicito de Python (desde 0)
* `.reset_index(drop=True)` = Cuando tiene un índice ya de números por ejemplo, para que no me cree una columna con los índices anteriores donde la etiqueta de la columna se llame 0 y si hay mas 0,1,2,etc, pongo `(drop=True)` adentro de reset_index. Y solo resetea los índices sin crear una nueva columna con los índices anteriores.

`pd.concat().reset_index(drop=True)` > Para lograr esto podría usarse también = `pd.concat([x, y], ignore_index=True)`

In [13]:
ser1 = pd.Series(['A', 'B', 'C'], index=[1, 2, 3])
print(ser1)
ser2 = pd.Series(['D', 'E', 'F'], index=[1, 2, 3])
print(ser2)
pd.concat([ser1, ser2], axis = 0).reset_index(drop=True)

1    A
2    B
3    C
dtype: object
1    D
2    E
3    F
dtype: object


0    A
1    B
2    C
3    D
4    E
5    F
dtype: object

También funciona para concatenar objetos de mayor dimensión, como ``DataFrame``s:

In [20]:
df1 = make_df('AB', [1, 2])
df2 = make_df('AB', [1, 2])

display('df1','df2','pd.concat([df1, df2], axis=1)')

df1
    A   B
1  A1  B1
2  A2  B2

df2
    A   B
1  A1  B1
2  A2  B2

pd.concat([df1, df2], axis=1)
    A   B   A   B
1  A1  B1  A1  B1
2  A2  B2  A2  B2

Por defecto, la concatenación se realiza por filas dentro del ``DataFrame`` (es decir, ``axis=0``).
Al igual que ``np.concatenate``, ``pd.concat`` permite especificar un eje a lo largo del cual se producirá la concatenación.

Considere el siguiente ejemplo:

In [23]:
df3 = make_df('AB', [0, 1])
df4 = make_df('CD', [0, 1])
display('df3', 'df4', "pd.concat([df3, df4], axis=0)", "pd.concat([df3, df4], axis=1)")

df3
    A   B
0  A0  B0
1  A1  B1

df4
    C   D
0  C0  D0
1  C1  D1

pd.concat([df3, df4], axis=0)
     A    B    C    D
0   A0   B0  NaN  NaN
1   A1   B1  NaN  NaN
0  NaN  NaN   C0   D0
1  NaN  NaN   C1   D1

pd.concat([df3, df4], axis=1)
    A   B   C   D
0  A0  B0  C0  D0
1  A1  B1  C1  D1

>>> ### **`pd.concat()` E INDICES DUPLICADOS**

Una diferencia importante entre ``np.concatenate`` y ``pd.concat`` es que **la concatenación en Pandas *PRESERVA LOS ÍNDICES***, ¡incluso si el resultado tiene índices duplicados!
Considera este sencillo ejemplo:

In [44]:
x = make_df('AB', [0, 1])
y = make_df('AB', [2, 3])
y.index = x.index  # ¡haz índices duplicados!
display('x', 'y', 'pd.concat([x, y], axis = 0)')

,A,B
0,A0,B0
1,A1,B1
,A,B
0,A2,B2
1,A3,B3
,A,B
0,A0,B0
1,A1,B1
0,A2,B2
1,A3,B3


* El **PROBLEMA** con dicho resultado, es si quiero llamar a algún valor que tenga índice 0, me devolverá dos valores y no puedo especificar cuál quiero. Salvo con `.head(1)` para que me muestre el de arriba por ejemplo, pero no es práctico. 

In [40]:
c = pd.concat([x, y], axis = 0)

In [41]:
c['A'][0]

0    A0
0    A2
Name: A, dtype: object

Observe los índices repetidos en el resultado.
Aunque esto es válido dentro de ``DataFrame``s, el resultado es a menudo indeseable.
``pd.concat()`` nos da algunas maneras de manejarlo.

>> #### **Captura de las repeticiones de INDICES como error** > **``verify_integrity=``**

Si quieres simplemente verificar que los índices en el resultado de ``pd.concat()`` no se solapan, puedes especificar la bandera ``verify_integrity``.
Con `True`, la concatenación lanzará una excepción si hay índices duplicados.
He aquí un ejemplo, en el que para mayor claridad capturaremos e imprimiremos el mensaje de error:

In [43]:
try:
    pd.concat([x, y], verify_integrity=True)
except ValueError as e:
    print("ValueError:", e)

ValueError: Indexes have overlapping values: Index([0, 1], dtype='int64')


>> #### **Ignorar el índice** > **``ignore_index=``**

- A veces, el índice en sí no importa y se prefiere ignorarlo, para que NO haya coincidencia de índices.
- Es similiar al método reset_index + drop=True = `pd.concat().reset_index(drop=True)` > Reseteaba los índices creando una columna con los anteriores, pero con drop=True, no creaba esta columna sino que directamente reseteaba los índices y eliminaba los originales. 

Esta opción puede especificarse utilizando el indicador ``ignore_index=True``.
Si se establece en `True`, la concatenación creará un nuevo índice desde el 0, para la ``Serie`` resultante sin repetidos. 

In [46]:
display('x', 'y', 'pd.concat([x, y], ignore_index=True)')

,A,B
0,A0,B0
1,A1,B1
,A,B
0,A2,B2
1,A3,B3
,A,B
0,A0,B0
1,A1,B1
2,A2,B2
3,A3,B3


>> #### **Añadir claves MultiIndex** > **``keys=``**

Otra opción es utilizar la opción ``keys`` para especificar una etiqueta general para cada DataFrame que se concatena; el resultado será una serie indexada jerárquicamente que contendrá los datos:

In [47]:
display('x', 'y', "pd.concat([x, y], keys=['x', 'y'])")

x
    A   B
0  A0  B0
1  A1  B1

y
    A   B
0  A2  B2
1  A3  B3

pd.concat([x, y], keys=['x', 'y'])
      A   B
x 0  A0  B0
  1  A1  B1
y 0  A2  B2
  1  A3  B3

El resultado es un ``DataFrame`` con múltiples índices, y podemos utilizar las herramientas comentadas en [Hierarchical Indexing](05_Hierarchical-Indexing.ipynb) para transformar estos datos en la representación que nos interesa.

>> ### **Concatenación con uniones >  `join='outer'` y  `join='inner'`**

Considera la concatenación de los siguientes dos ``DataFrame``s, que tienen algunas (¡pero no todas!) columnas en común, por lo que los valores faltantes seran rellenados con `NaN`:

In [55]:
df5 = make_df('ABC', [1, 2])
df6 = make_df('BCD', [3, 4])
# display(df5)
# display(df6)
display('df5', 'df6', 'pd.concat([df5, df6])')

df5
    A   B   C
1  A1  B1  C1
2  A2  B2  C2

df6
    B   C   D
3  B3  C3  D3
4  B4  C4  D4

pd.concat([df5, df6])
     A   B   C    D
1   A1  B1  C1  NaN
2   A2  B2  C2  NaN
3  NaN  B3  C3   D3
4  NaN  B4  C4   D4

- **`join='outer'`** = ES LA QUE VIENE POR DEFECTO SI NO PONGO NADA >> Devuelve **todas las filas de ambas tablas**, rellenando con **valores nulos** donde **no haya coincidencias**. Es la unión de ambos conjuntos de datos. Si un valor está presente en una tabla pero no en la otra, aún estará presente en el resultado.
- **`join='inner'`** = Devuelve las **filas que tienen valores que existen en ambas tablas**, o sea las **coincidencias**, es decir, la intersección de los DataFrames. Si un valor está presente en una tabla pero no en la otra, ese valor no estará en el resultado.Es donde haya coincidencias. 

En el ejemplo siguiente, devuelve solo las columnas coincidentes, o sea, B y C >>

In [56]:
display('df5', 'df6',
        "pd.concat([df5, df6], join='inner')")

df5
    A   B   C
1  A1  B1  C1
2  A2  B2  C2

df6
    B   C   D
3  B3  C3  D3
4  B4  C4  D4

pd.concat([df5, df6], join='inner')
    B   C
1  B1  C1
2  B2  C2
3  B3  C3
4  B4  C4